# Problem statement

## Question

How can [fivethirtyeight.com/politics](https://fivethirtyeight.com/politics/) get more traffic to their articles, without compromising their political neutrality and reputation for factually correct content?

## Background



# Stakeholders

News has become more polarized and sensationalized in recent years, all in the name of more clicks.  This data analysis could provide some insight into what kind of articles get more traffic, without news organizations having to sacrifice their integrity.


# Key performance indicators (KPIs)

- Number of comments an article gets, relative to how long it's been posted

# Data gathering

We scrape headlines from the [Five Thirty Eight politics features pages](https://fivethirtyeight.com/politics/features/) from the year to date.  For each headline, we store the title, author, date, tags, and number of comments.